In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itrf" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:43<00:00,  5.68s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  42%|████▏     | 11/26 [00:00<00:00, 107.92batch/s, loss=0.272]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9172 | AUPR: 0.6548 | Acc: 95.34%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9199 | AUPR: 0.6630 | Acc: 95.29%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9219 | AUPR: 0.6707 | Acc: 95.36%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9174 | AUPR: 0.6564 | Acc: 95.27%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9202 | AUPR: 0.6622 | Acc: 95.32%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9216 | AUPR: 0.6665 | Acc: 95.40%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9219 | AUPR: 0.6707 | Acc: 95.36%
Internal test: AUC: 0.9303 | AUPR: 0.6948 | Acc: 95.30%
External test: AUC: 0.9214 | AUPR: 0.6669 | Acc: 95.16%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC


model(kernel=linear, C=0.01)
Valid:         AUC: 0.9176 | AUPR: 0.6376 | Acc: 95.18%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9181 | AUPR: 0.6373 | Acc: 95.17%
model(kernel=linear, C=1)
Valid:         AUC: 0.9169 | AUPR: 0.6343 | Acc: 95.16%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9181 | AUPR: 0.6373 | Acc: 95.17%
Internal test: AUC: 0.9265 | AUPR: 0.6821 | Acc: 95.09%
External test: AUC: 0.9159 | AUPR: 0.6269 | Acc: 94.80%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7489 | AUPR: 0.4302 | Acc: 94.81%
model(n_neighbors=5)
Valid:         AUC: 0.7803 | AUPR: 0.4716 | Acc: 95.01%
model(n_neighbors=7)
Valid:         AUC: 0.7989 | AUPR: 0.4972 | Acc: 95.12%
best model(n_neighbors=7)
Valid:         AUC: 0.7989 | AUPR: 0.4972 | Acc: 95.12%
Internal test: AUC: 0.8053 | AUPR: 0.5516 | Acc: 95.11%
External test: AUC: 0.8080 | AUPR: 0.5103 | Acc: 94.91%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8764 | AUPR: 0.4220 | Acc: 84.07%
best model()
Valid:         AUC: 0.8764 | AUPR: 0.4220 | Acc: 84.07%
Internal test: AUC: 0.8838 | AUPR: 0.4488 | Acc: 84.11%
External test: AUC: 0.8746 | AUPR: 0.4025 | Acc: 82.87%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9188 | AUPR: 0.6543 | Acc: 95.25%
model(C=10.0)
Valid:         AUC: 0.9184 | AUPR: 0.6532 | Acc: 95.22%
model(C=100.0)
Valid:         AUC: 0.9177 | AUPR: 0.6529 | Acc: 95.22%
best model(C=1.0)
Valid:         AUC: 0.9188 | AUPR: 0.6543 | Acc: 95.25%
Internal test: AUC: 0.9276 | AUPR: 0.6882 | Acc: 95.23%
External test: AUC: 0.9150 | AUPR: 0.6375 | Acc: 94.97%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8744 | AUPR: 0.5579 | Acc: 85.21%
model(criterion=log_loss)
Valid:         AUC: 0.8805 | AUPR: 0.5646 | Acc: 85.59%
model(criterion=entropy)
Valid:         AUC: 0.8805 | AUPR: 0.5646 | Acc: 85.59%
best model(criterion=log_loss)
Valid:         AUC: 0.8805 | AUPR: 0.5646 | Acc: 85.59%
Internal test: AUC: 0.8804 | AUPR: 0.6087 | Acc: 87.13%
External test: AUC: 0.8793 | AUPR: 0.5746 | Acc: 87.22%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9148 | AUPR: 0.6599 | Acc: 89.70%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9154 | AUPR: 0.6602 | Acc: 89.91%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9160 | AUPR: 0.6602 | Acc: 89.98%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9148 | AUPR: 0.6605 | Acc: 89.91%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9159 | AUPR: 0.6610 | Acc: 90.08%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9162 | AUPR: 0.6615 | Acc: 90.31%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9148 | AUPR: 0.6605 | Acc: 89.91%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9159 | AUPR: 0.6610 | Acc: 90.08%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9162 | AUPR: 0.6615 | Acc: 90.31%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9162 | AUPR: 0.6615 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9186 | AUPR: 0.6521 | Acc: 95.24%
model(criterion=squared_error)
Valid:         AUC: 0.9191 | AUPR: 0.6537 | Acc: 95.23%
best model(criterion=squared_error)
Valid:         AUC: 0.9191 | AUPR: 0.6537 | Acc: 95.23%
Internal test: AUC: 0.9268 | AUPR: 0.6955 | Acc: 95.45%
External test: AUC: 0.9193 | AUPR: 0.6605 | Acc: 95.12%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9163 | AUPR: 0.6348 | Acc: 95.06%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9157 | AUPR: 0.6365 | Acc: 95.08%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9151 | AUPR: 0.6356 | Acc: 95.11%
best model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9163 | AUPR: 0.6348 | Acc: 95.06%
Internal test: AUC: 0.9213 | AUPR: 0.6765 | Acc: 95.16%
External test: AUC: 0.9131 | AUPR: 0.6180 | Acc: 94.76%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)